<a href="https://colab.research.google.com/github/krishna1194/PySpark_SQL/blob/main/PySpark_SQL_Getting_Started.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PySpark SQL - Getting Started Notebook

This notebook provides an example for installing Spark dependencies and a simple "getting started" syntax for PySpark SQL.

NOTE: This notebook is designed to be used with Google's Colab notebook and the Python 3 runtime.

## Install Spark Dependencies

In [ ]:
# Install Spark dependencies
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!rm spark-3.2.3-bin-hadoop3.2.tgz
!wget --no-cookies --no-check-certificate https://dlcdn.apache.org/spark/spark-3.2.3/spark-3.2.3-bin-hadoop3.2.tgz
!tar zxvf spark-3.2.3-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark==3.2.3
!ls -al | grep spark

## Set environment variables within Pyspark

In [2]:
# Set up required environment variables
import os
os.environ["JAVA_HOME"]  = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.3-bin-hadoop3.2"

## Download Data

In [8]:
!wget https://raw.githubusercontent.com/zaratsian/iaa-2023/master/session_02/bikeshare_station_info.csv
!wget https://raw.githubusercontent.com/zaratsian/iaa-2023/master/session_02/bikeshare_trips.csv

--2023-03-12 00:59:44--  https://raw.githubusercontent.com/zaratsian/iaa-2023/master/session_02/bikeshare_station_info.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80415 (79K) [text/plain]
Saving to: ‘bikeshare_station_info.csv’

bikeshare_station_i 100%[===================>]  78.53K  --.-KB/s    in 0.01s   

2023-03-12 00:59:44 (6.31 MB/s) - ‘bikeshare_station_info.csv’ saved [80415/80415]

--2023-03-12 00:59:44--  https://raw.githubusercontent.com/zaratsian/iaa-2023/master/session_02/bikeshare_trips.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiti

## Import Python and PySpark Libraries

In [4]:
import datetime
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import monotonically_increasing_id, col, expr, when, concat, lit, udf, split
from pyspark.ml.linalg import Vectors
from pyspark.ml.regression import GBTRegressor, LinearRegression, GeneralizedLinearRegression
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import VectorIndexer, VectorAssembler, StringIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

## Initialize Spark Session

In [5]:
spark = SparkSession.builder.appName("Bikesharing SparkSQL").master("local[*]").getOrCreate()

## Read CSV into Spark

In [9]:
bikeshare_trips = spark.read.load('bikeshare_trips.csv', format="csv", header=True, inferSchema=True)

In [12]:
# Display first few records
bikeshare_trips.show(5, truncate=False)

+--------------------+------------+-----------------------+-----------------------+----------------+-----------------------+--------------------+--------------+-----------+--------+---------------+-------------------+----------------------+-----------------------+--------------------+---------------------+-----------------+-------------+-----------------------+------------------------------+----------------------------+
|trip_id             |duration_sec|start_date             |start_station_name     |start_station_id|end_date               |end_station_name    |end_station_id|bike_number|zip_code|subscriber_type|c_subscription_type|start_station_latitude|start_station_longitude|end_station_latitude|end_station_longitude|member_birth_year|member_gender|bike_share_for_all_trip|start_station_geom            |end_station_geom            |
+--------------------+------------+-----------------------+-----------------------+----------------+-----------------------+--------------------+-------

In [13]:
# Execute Sample SparkSQL query
bikeshare_trips.createOrReplaceTempView("bikeshare_trips1")
spark.sql("SELECT subscriber_type, count(*) as count FROM bikeshare_trips1 group by subscriber_type order by count desc").show(truncate=False)

+---------------+-----+
|subscriber_type|count|
+---------------+-----+
|Subscriber     |13685|
|Customer       |2315 |
+---------------+-----+

